# **The Battle of Neighborhoods: Opening a Restaurant Across Horizon** 
#### **Building models for segmentation of neighborhoods to find favorable locations for starting a fast food joint in the city of Toronto.**
by: [Usman Khan](https://www.linkedin.com/in/usman-khan-458755163/)

### **Description of Data**

The data required are mentioned below:

* 1st Data:
Recent most updated record of traffic signal -  vehicle and pedestrian volumes in Toronto. This data is typically collected between 7:30 a.m. and 6:00 p.m at intersections where there are traffic signals.
    
* 2nd Data: 
The list of neighborhoods in Toronto represented by postal codes and their boroughs. We will be using the Geocoder Python package in order to retrieve the postal code's coordinates.

* 3rd Data: 
The most common venues of a given neighborhood in Toronto. This information is stored inside Foursquare Location Data, and we will use Foursquare API in order to access it.

To recap, we will use the 1st Data to analyze the pedestrian/vehicle volume. Then, we load the 2nd Data to obtain the exact coordinates for each neighborhood based on their respective postal code, allowing us to explore and map the city. By using those coordinates and Foursquare credentials, we will access the 3rd Data source through Foursquare API,  and retrieve the popular venues along with their details, especially for restaurants irrespective of their cuisine.